In [1]:
from geopy import Nominatim # convert an address into latitude and longitude values

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [3]:
import geocoder


In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Bangalore'
website_url = requests.get(url).text
soup = BeautifulSoup(website_url,'lxml')

In [5]:
t = soup.find('table',{'class':'wikitable sortable'})

In [6]:
t = t.find_all('a')

In [7]:
area=[]
for i in t:
    area.append(str(i.get('title')).replace(', Bangalore',''))

In [8]:
n = pd.DataFrame({'Area':area})
n = n[n['Area']!='None']
n.reset_index(drop=True)
n

,Area
0,Bangalore Cantonment
1,British Raj
2,Richmond Town
3,Austin Town
4,Murphy Town
5,Fraser Town
6,Cox Town
7,Domlur
9,Embassy GolfLinks Business Park
10,Hindustan Aeronautics Limited


In [9]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Bangalore, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [10]:
coords = [ get_latlng(neighborhood) for neighborhood in n["Area"].tolist() ]

In [11]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [12]:
n = n.join(df_coords)

In [16]:
n.reset_index(drop=True)
n=n.head(25)

In [17]:
address = 'Bangalore,India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [18]:
# create map of Toronto using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(n['Latitude'], n['Longitude'], n['Area']):
        label = '{}'.format(neighborhood)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [19]:
CLIENT_ID = 'EEN43PDYEJFHILBS0VAQESWL5TBEHM0ZDFBIAASSI5TUHHQ5' # your Foursquare ID
CLIENT_SECRET = 'S0JPJP4QKW2WB3NOLOO2S51C35K5AIXRHBEDKDOW4NLEHRUE' # your Foursquare Secret
VERSION = '20200410' # Foursquare API version

In [20]:
radius = 1500
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(n['Latitude'], n['Longitude'], n['Area']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [21]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1560, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Bangalore Cantonment,12.97568,77.60539,M.G Road Boulevard,12.975771,77.603979,Plaza
1,Bangalore Cantonment,12.97568,77.60539,Blossom Book House,12.975042,77.604813,Bookstore
2,Bangalore Cantonment,12.97568,77.60539,The 13th Floor,12.975364,77.604995,Lounge
3,Bangalore Cantonment,12.97568,77.60539,Matteo,12.974496,77.607115,Café
4,Bangalore Cantonment,12.97568,77.60539,Starbucks,12.974436,77.607308,Coffee Shop


In [22]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(1560, 162)


,Neighborhoods,ATM,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Bed & Breakfast,Bengali Restaurant,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Camera Store,Campground,Candy Store,Capitol Building,Chaat Place,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gastropub,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Historic Site,Hockey Arena,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,New American Restaurant,Nightclub,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Real Estate Office,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Salad Place,Sandwich Place,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Track Stadium,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Watch Shop,Wine Bar,Wine Shop,Women's Store
0,Bangalore Cantonment,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Bangalore Cantonment,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Bangalore Cantonment,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Bangalore Cantonment,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Bangalore Cantonment,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [23]:

# set number of clusters
kclusters = 5

x_data = kl_onehot.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(x_data)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]
l = kmeans.labels_

In [24]:
l = pd.DataFrame({'Category':l})

In [25]:
banglore_data = n.join(l)

In [26]:
banglore_data.head()

,Area,Latitude,Longitude,Category
0,Bangalore Cantonment,12.975680,77.60539,0
1,British Raj,13.105087,77.60762,0
2,Richmond Town,12.962340,77.60123,0
3,Austin Town,12.963480,77.61297,2
4,Murphy Town,12.980030,77.62776,0


Indranagar is one of the most commercial and crowded area 

In [27]:
banglore_data[banglore_data["Area"]=='Indiranagar']

,Area,Latitude,Longitude,Category
11,Indiranagar,12.906541,77.573181,0


In [29]:
print("Possible Locations are :")
banglore_data[banglore_data['Category']==0]

Possible Locations are :


,Area,Latitude,Longitude,Category
0,Bangalore Cantonment,12.975680,77.605390,0
1,British Raj,13.105087,77.607620,0
2,Richmond Town,12.962340,77.601230,0
4,Murphy Town,12.980030,77.627760,0
5,Fraser Town,12.998940,77.612760,0
6,Cox Town,12.995940,77.626440,0
7,Domlur,12.943290,77.656020,0
9,Embassy GolfLinks Business Park,12.954980,77.663290,0
11,Indiranagar,12.906541,77.573181,0
13,100 Feet Road,13.006322,77.568416,0
